# Import libraries

In [46]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Load Data Set

In [48]:
def load_coffee_data():
    """ Creates a coffee roasting data set.
        roasting duration: 12-15 minutes is best
        temperature range: 175-260C is best
    """
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [49]:
X, y = load_coffee_data()
print(X.shape, y.shape)

(200, 2) (200, 1)


# Normalization of Input

In [50]:
norm_l = tf.keras.layers.Normalization(axis=-1)
norm_l.adapt(X)
Xn = norm_l(X)

# Tile Data to increase the Data Set

In [56]:
Xt = np.tile(Xn, (1000, 1))
yt = np.tile(y, (1000, 1))
print(Xt.shape, yt.shape)

(200000, 2) (200000, 1)


#  Model Structure

In [61]:
tf.random.set_seed(1234)
model = Sequential([
    tf.keras.Input(shape=(2,)),
    Dense(3, activation='sigmoid', name='layer1'),
    Dense(1, activation='sigmoid', name='layer2')
])

In [62]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 3)                 9         
                                                                 
 layer2 (Dense)              (None, 1)                 4         
                                                                 
Total params: 13
Trainable params: 13
Non-trainable params: 0
_________________________________________________________________


# Initial Parameters

In [66]:
W1, b1 = model.get_layer('layer1').get_weights()
W2, b2 = model.get_layer('layer1').get_weights()
print(f"W1{W1.shape}\n", W1, f"\nb1{b1.shape}\n", b1)
print(f"W2{W2.shape}\n", W1, f"\nb2{b2.shape}\n", b2)

W1(2, 3)
 [[ 0.42850006 -1.083289   -0.25739557]
 [ 0.45483613 -0.41253448 -0.54406536]] 
b1(3,)
 [0. 0. 0.]
W2(2, 3)
 [[ 0.42850006 -1.083289   -0.25739557]
 [ 0.45483613 -0.41253448 -0.54406536]] 
b2(3,)
 [0. 0. 0.]


# Train the Model

In [67]:
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
)

model.fit(
    Xt,Yt,            
    epochs=10,
)

Epoch 1/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.1717
Epoch 2/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.1004
Epoch 3/10
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0260
Epoch 4/10
6250/6250 [==============================] - 7s 1ms/step - loss: 0.0138
Epoch 5/10
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0090
Epoch 6/10
6250/6250 [==============================] - 6s 979us/step - loss: 0.0062
Epoch 7/10
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0043
Epoch 8/10
6250/6250 [==============================] - 6s 1ms/step - loss: 0.0030
Epoch 9/10
6250/6250 [==============================] - 6s 968us/step - loss: 0.0021
Epoch 10/10
6250/6250 [==============================] - 6s 967us/step - loss: 0.0015


# Optimized Parameters

In [78]:
W1, b1 = model.get_layer('layer1').get_weights()
W2, b2 = model.get_layer('layer2').get_weights()
print(f"W1{W1.shape}\n", W1, f"\nb1{b1.shape}\n", b1)
print(f"W2{W2.shape}\n", W2, f"\nb2{b2.shape}\n", b2)

W1(2, 3)
 [[ 14.612482     0.18546233 -10.998361  ]
 [ 12.107305    10.479361    -0.30711606]] 
b1(3,)
 [  1.9996973  12.646509  -11.82469  ]
W2(3, 1)
 [[-47.27569 ]
 [ 45.369507]
 [-55.433495]] 
b2(1,)
 [-14.098207]


In [80]:
W1 = np.array([
    [ 14.612482,     0.18546233, -10.998361  ],
    [ 12.107305,    10.479361,    -0.30711606]
])
b1 = np.array([  1.9996973,  12.646509,  -11.82469  ])
W2 = np.array([
    [-47.27569 ],
    [ 45.369507],
    [-55.433495]
])
b2 = np.array([-14.098207])

# Set new Parameters to the Model

In [81]:
model.get_layer("layer1").set_weights([W1, b1])
model.get_layer("layer2").set_weights([W2, b2])

# Prediction

In [82]:
X_test = np.array([
    [200,13.9],  # postive example
    [200,17]])   # negative example
X_testn = norm_l(X_test)
predictions = model.predict(X_testn)
print("predictions = \n", predictions)

1/1 [==============================] - 0s 35ms/step
predictions = 
 [[9.9400502e-01]
 [1.0914961e-07]]


In [83]:
yhat = np.zeros_like(predictions)
for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        yhat[i] = 1
    else:
        yhat[i] = 0
print(f"decisions = \n{yhat}")

decisions = 
[[1.]
 [0.]]


In [84]:
yhat = (predictions >= 0.5).astype(int)
print(f"decisions = \n{yhat}")

decisions = 
[[1]
 [0]]
